In [1]:
import sys
sys.path.append("..")
from pathlib import Path
from utils.visualise import get_ocr_stat, display_ocr, display_folder
from utils.config import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_data_train = "../data/train"
path_data_val = "../data/val"
path_data_test = "../data/test"

path_folders_train = "../data/train_folders.txt"
path_folders_val = "../data/val_folders.txt"
path_folders_test = "../data/test_folders.txt"

In [3]:
folders_test = np.array([line.split() for line in open(path_folders_test).readlines(  )])
folders_test

array([['ffxy0032', '1'],
       ['fflk0052', '1'],
       ['ffdh0117', '1'],
       ...,
       ['ffbb0027', '5000'],
       ['ffnv0005', '5000'],
       ['ffbx0075', '5000']], dtype='<U8')

In [4]:
pages_test_ocr = np.array([(path.stem.split('-') if len(path.stem.split('-'))==2 else [path.stem, '0']) for path in Path(path_data_test).glob("**/*.tsv")])
pages_test_img = np.array([(path.stem.split('-') if len(path.stem.split('-'))==2 else [path.stem, '0']) for path in Path(path_data_test).glob("**/*.jpg")])
assert pages_test_ocr.shape==pages_test_img.shape, f"number of pages in ocr and images not match"

folders_test = np.array([line.split() for line in open(path_folders_test).readlines(  )])
assert np.unique(folders_test[:,0]).shape[0]==np.unique(pages_test_ocr[:,0]).shape[0], f"number of documents in files and txt not match"

print("Test set:")
print(f"   found {np.unique(folders_test[:,0]).shape[0]} documents, {pages_test_ocr.shape[0]} pages, {np.unique(folders_test[:,1]).shape[0]} folders")


Test set:
   found 2238 documents, 6237 pages, 5000 folders


In [5]:
pages_test_img

array([['ffbb0002', '0'],
       ['ffbb0027', '0'],
       ['ffbb0118', '0'],
       ...,
       ['ffyy0191', '0'],
       ['ffyy0191', '1'],
       ['ffyy0212', '0']], dtype='<U8')

In [6]:
df = pd.DataFrame({"id": pages_test_img[:,0], "page_num": pages_test_img[:,1]})
df["stem"] = df.id + '-' + df.page_num
df["label"] = (df["page_num"].astype(int)==0).astype(int)

In [7]:
df1 = pd.DataFrame({"id": folders_test[:,0], "folder_id": folders_test[:,1]})

In [8]:
df1 = df1.merge(df, on = "id", how = "left")

In [9]:
folder = df1[["folder_id", "id"]].groupby("folder_id").count()
folder.rename(columns = {"id":"merged_num_of_pages"}, inplace = True)
df1 = df1.merge(folder, on = "folder_id")

In [10]:
df1.head(50)

,id,folder_id,page_num,stem,label,merged_num_of_pages
0,ffxy0032,1,0,ffxy0032-0,1,19
1,ffxy0032,1,1,ffxy0032-1,0,19
2,ffxy0032,1,2,ffxy0032-2,0,19
3,ffxy0032,1,3,ffxy0032-3,0,19
4,fflk0052,1,0,fflk0052-0,1,19
5,ffdh0117,1,0,ffdh0117-0,1,19
6,ffcd0018,1,0,ffcd0018-0,1,19
7,ffxy0005,1,0,ffxy0005-0,1,19
8,ffxy0005,1,1,ffxy0005-1,0,19
9,ffmd0041,1,0,ffmd0041-0,1,19


In [14]:
folder.mean()

merged_num_of_pages    30.4286
dtype: float64

In [12]:
df1.to_csv("full_model_prediction.csv")